In [ ]:
from scipy.ndimage import rotate
import pandas as pd 
import numpy as np 
import math
# libraries for data visualization
import matplotlib.pyplot as plt
# libraries for model selection
import xgboost as xgb
# libraries for model evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
#library to read matlab files
from scipy.io import loadmat
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV


<h1>Load and Parse Data</h1>

In [ ]:
#Import the MNIST.mat file
mat_data = loadmat('/Users/deep/Desktop/ML_Project/MNIST.mat')
# Print the keys of the MATLAB file to understand what data it contains
print(mat_data.keys())

pixel_features = mat_data['train_fea']
pixel_labels = mat_data['train_gnd'].ravel()

pixel_test_features = mat_data['test_fea']
pixel_test_labels = mat_data['test_gnd'].ravel()

#Normalize the pixel features
pixel_features = pixel_features/255
pixel_test_features = pixel_test_features/255
